# Bank Transactions Pipeline
## All transactions using Monzo and other cards through Google Sheets Manual Transactions for 2018-2021

In [10]:
import os
import gspread
import monzo as m
from monzo.handlers.echo import Echo
from monzo.endpoints.transaction import Transaction
from monzo.endpoints.balance import Balance
from monzo.endpoints.pot import Pot
from monzo.handlers.filesystem import FileSystem
from monzo.authentication import Authentication
from monzo.exceptions import MonzoAuthenticationError, MonzoServerError
from db import FinancesDb
import datetime
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from categories import cat_dict, columns_dict
from json import dumps, loads
pd.set_option('display.max_rows', 5000)
import warnings
warnings.filterwarnings("ignore")

### Authenticaion

In [11]:
client_id = os.environ.get('MONZO_CLIENT_ID')
client_secret = os.environ.get('MONZO_CLIENT_SECRET')
redirect_uri = 'http://127.0.0.1/monzo'  
start_date = datetime.datetime.now() - datetime.timedelta(89)

with open('/Users/eliasbenaddouidrissi/Desktop/projects/monzo_dashboard/monzo_dashboard/tokens', 'r') as tokens:
    content = loads(tokens.read())

monzo = Authentication(client_id=client_id, client_secret=client_secret,
                     redirect_url=redirect_uri, access_token=content['access_token'],
                     refresh_token=content['refresh_token'], access_token_expiry=content['expiry'])

handler = FileSystem('/Users/eliasbenaddouidrissi/Desktop/projects/monzo_dashboard/monzo_dashboard/tokens') 
monzo.register_callback_handler(handler)

### Importing new Monzo Transactions

In [12]:
fetched_transactions = Transaction.fetch(auth=monzo, account_id='acc_00009Uu4ltUzxjOvYjP87d')

transactions_dict = {}

for index, row in enumerate(fetched_transactions):
    transaction_data = Transaction.fetch_single(auth=monzo, transaction_id=fetched_transactions[index].transaction_id)

    id = transaction_data.transaction_id
    scheme = transaction_data.scheme
    date = transaction_data.created
    description = transaction_data.description
    amount = transaction_data.amount
    category = transaction_data.category
    categories = transaction_data.categories
    counterparty = transaction_data.counterparty
    decline_reason = transaction_data.decline_reason
    merchant = transaction_data.merchant
    meta = transaction_data.metadata

    transactions_dict[index] = id, date, description, amount, category, categories, counterparty, scheme, decline_reason, merchant, meta

transactions_df = pd.DataFrame(transactions_dict).T
transactions_df.rename(columns ={0:'id', 1:'date', 2:'description',
 3:'amount', 4:'category', 5:'categories', 6:'counterparty', 7:'scheme',
  8:'decline_reason', 9:'merchant_info', 10:'meta'}, inplace=True)
transactions_df['decline'] = np.where(transactions_df['decline_reason'] == '', 0, 1 )

In [13]:
merchant_info = pd.json_normalize(transactions_df['merchant_info'])
metadata = pd.json_normalize(transactions_df['meta'])
multiple_categories = pd.json_normalize(transactions_df['categories'])

transactions_merchant = transactions_df.merge(merchant_info, left_index=True, right_index=True)
transactions_meta = transactions_merchant.merge(metadata, left_index=True, right_index=True)
transactions_multiple_categories = transactions_meta.merge(multiple_categories, left_index=True, right_index=True)

transactions = transactions_multiple_categories.drop(['merchant_info', 'meta', 'categories', 'counterparty'], axis=1)
transactions['category_x'].replace(cat_dict, inplace=True)
transactions['amount'].apply(lambda x: (-1*(x/100)))
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')

transactions.reset_index(drop=True, inplace=True)
transactions.rename(columns=columns_dict, inplace=True)       
transactions.drop(['updated', 'created'], axis=1)

### Manual Imported Transactions

In [14]:
sa = gspread.service_account().open_by_url("https://docs.google.com/spreadsheets/d/11y1xCKEY8x_bHGiD8H9Kn3ytVSFtc_e-CF2t7Q8qQFQ/edit#gid=1283738573").worksheet("Form Responses")
df_manual_trans = pd.DataFrame(sa.get_all_records()).rename(columns={'Timestamp': 'date', 'What did you purchase/deposit?': 'merchant_description', 'What category does it fall into?': 'category', 'What was the amount?': 'amount'})
# converting date column to datetime object
df_manual_trans.date = pd.to_datetime(df_manual_trans.date)

In [15]:
cat_dict = {
    'Groceries': 'groceries',
    'Transport': 'transport',
    'Eating Out': 'eating_out',
    'Entertainment': 'entertainment',
    'Phone': 'bills',
    'Community Fibre': 'bills',
    'iCloud Storage': 'subscriptions',
    'Gifts': 'gifts',
    'Netflix': 'subscriptions',
    'Amazon Prime': 'subscriptions',
    'Fees': 'fees',
    'Gym': 'gym',
    'Pet Food': 'pets',
    'Tech': 'shopping',
    'Clothes': 'clothes',
    'Healthcare': 'healthcare',
    'Haircut': 'haircut',
    'NHS Prescription': 'subscriptions',
    'Stocks': 'stocks',
    'Crypto': 'crypto',
    'Holidays': 'holidays',
    'Insurance': 'bills',
    'Learning': 'education',
    'Home': 'home',
    'Income': 'income',
    'Capgemini Invent': 'income',
    'Samsung': 'income',
    'Vanguard': 'isa',
    'Club Lloyds': 'savings',
    'HSBC': 'savings',
    'Marcus': 'savings',
    'Interest': 'savings',
    'Subscriptions': 'subscriptions',

}

df_manual_trans['category'].replace(cat_dict, inplace=True)
df_manual_trans.rename(columns={'merchant_description':'description'}, inplace=True)

### All transactions (Monzo + manual)

In [16]:
sorted(list(df_manual_trans.category.unique()))

['bills',
 'clothes',
 'crypto',
 'eating_out',
 'education',
 'entertainment',
 'fees',
 'gifts',
 'groceries',
 'gym',
 'haircut',
 'healthcare',
 'holidays',
 'home',
 'income',
 'isa',
 'pets',
 'savings',
 'shopping',
 'stocks',
 'subscriptions',
 'transport']

In [18]:
sorted(list(transactions.category.unique()))

['bills',
 'clothes',
 'crypto',
 'eating_out',
 'entertainment',
 'expenses',
 'fees',
 'general',
 'gifts',
 'groceries',
 'gym',
 'haircut',
 'healthcare',
 'holidays',
 'home',
 'income',
 'pets',
 'refunds',
 'shopping',
 'subscriptions',
 'transfers',
 'transport',
 'withdrawals']

In [20]:
# adding id's to manual transactions
f, u = pd.factorize(df_manual_trans.date.values)
df_manual_trans = df_manual_trans.assign(id='tx_0000' + pd.util.testing.rands_array(18, u.size)[f])

#adding flag for monzo/manual
transactions = transactions.assign(origin = 'monzo')
df_manual_trans = df_manual_trans.assign(origin = 'manual')

df = pd.concat([transactions, df_manual_trans], axis=0, ignore_index=True)
df = df.sort_values(by='date').reset_index(drop=True)


### Insert all transactions to database

In [23]:
db = FinancesDb()

[2022-01-02 14:24:20] Database connection established to Finances


In [24]:
db.create('public.transactions', df=df)

[2022-01-02 14:24:22] Running SQL: exists
[2022-01-02 14:24:25] Table created and 2144 rows inserted into public.transactions
